In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

We define transform composition which will be used to preprocess each image.
We will just transform each image from its PIL image type to pytorch Tensor and then normalize it so the pixels with
larger value don't have bigger impact than those with the lower value. Same principle as transforming all attributes to the same scale.

In [3]:
transform = transforms.Compose([
    transforms.ToTensor(),
    lambda x: x/255.
])

We are now able to download the data. From the beginning, we will divide the data into two parts: training and testing, in order to evaluate the model's performance on unseen examples. Additionally, we will require a validation set. Typically, the optimal model is determined throughout the epochs using the validation set. Subsequently, we compare the models with varying hyperparameter values or entirely different algorithms, based on their performance on the test set.

In [4]:
full_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)

test_dataset = datasets.MNIST(root='./data', train = False, download = True, transform = transform)

train_size = int(0.6 * len(full_dataset))
val_size = len(full_dataset) - train_size

train_dataset, val_dataset = torch.utils.data.random_split(full_dataset, [train_size, val_size])

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



Now we can define dataloaders for our datasets which will handle loading data into the model.
We can try using different batch sizes on the train data loader which can lead to better performance because of the regularization effect that
minibatched gradients provide. On test and validation loaders we can use the biggest batch size that memory allows us because we are only testing the model not training it.

Also we are going to shuffle train data so we can give the model a better chance to not get stuck in the bad local minima. For the same reason as mentioned above we won't shuffle test and validation sets.

num_workers is set to 2 so the model uses multiple processes to load the data into the GPU device and minimzes the time that GPU is not used. Even though 2 is the usual number of workers the ideal value depends on the data we are working with and the batch size so we can look at it as the hyper parameter for the model training duration.

Last attribute pin_memory is set to True so that we remove one step in transitioning data from the host to the device and directly move the data from the pinned memory. This will also save us some time during the model training.




In [5]:
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2, pin_memory = True)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=128, shuffle=False, num_workers=2, pin_memory = True)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=2, pin_memory = True)







The "go to" architecture for all visual tasks are convolutional neural networks. Convolutional networks generate high quality visual embeddings that can then be fed into fully connected neural network (classification head). Behind the high quality of image embeddings is the CNNs ability to capture spatial relationships between pixels along with pooling layers which remove the impact of different positions of the digit in the image.

Although MNIST task is relatively simple one and we use a small neural network we will also use batch normalization which will result in more stable training and reduce overfitting. Benefits of the batch normalization are usually observered in really deep networks like Resnet or EffiecentNet and also in small GAN networks.

To show the simplicity of the MNIST dataset task we will also use small fully connected neural network to see how high can we get accuracy with a relatively primitive approach.

On the last layer of both networks we are going to left out the activation function because we will use metrics that expect the logits as the input.



In [6]:
import torch.nn as nn
import torch.nn.functional as F



class ConvModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels = 1, out_channels = 16, kernel_size = 3, stride =  1)
        self.bn1 = nn.BatchNorm2d(num_features = 16)
        self.mp1 = nn.MaxPool2d(kernel_size = 2)
        
        self.conv2 = nn.Conv2d(in_channels = 16, out_channels = 32, kernel_size = 3, stride =  1)
        self.bn2 = nn.BatchNorm2d(num_features = 32)
        self.mp2 = nn.MaxPool2d(kernel_size = 2)
        
        self.conv3 = nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size = 3, stride = 1)
        self.mp3 = nn.MaxPool2d(kernel_size = 2)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(64,100)
        self.fc2 = nn.Linear(100,10)
        
    def forward(self,x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = F.relu(x)
        x = self.mp1(x)
        
        x = self.conv2(x)
        x = self.bn2(x)
        x = F.relu(x)
        x = self.mp2(x)
        
        x = self.conv3(x)
        x = F.relu(x)
        x = self.mp3(x)
        
        x = self.flatten(x)
        
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x   
    
class FCModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(28*28,100)
        self.fc2 = nn.Linear(100,10)
        
    def forward(self,x):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)
        
        return x
        
        
        
        
    

In [10]:
conv_model = ConvModel()
fc_model = FCModel()


Now we can set up the optimizer algorithm which will be used during training. We choose widely used ADAM algorithm, best combination of the AdaGrad and RMSProp algorithms. Learning rate will be adapted with the optimizer so we can start relatively small.


We are going to check if the GPU device is available and if it is (it always should be) we are going to transfer our model weight tensors on it for faster training.

Our loss will be CrossEntropy loss which is usually used for classifcation tasks and pytorch CrossEntropy loss expects logits of the model so in a way now we are setting the last activation layer and it will be softmax which is used in cases when only one class is the right prediction. Reasons behind usage of logits is the way Pytorch CrossEntropyLoss utilizes the possiblity of combining loss function with the softmax layer. 

Also for the same reason we are going to use MulticlassAccuracy class. Usually it would be best to follow other metrics also like Precision, Recall and F-score which is more robust to specific tasks and disbalanced datasets but in this case we can rely on accuracy.


Furthermore we are building the loop for the model training.
Each epoch we go through the whole training set each iteration processing one mini batch, forward pass, backpropagation,
weight updates and metric updates. After each epoch we validate model on the validation set and keep the highest performing one.

In [12]:
from tqdm import tqdm
from copy import deepcopy
from torchmetrics.classification import MulticlassAccuracy

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def train_loop(model):
    
    #optimization algorithm
    LEARNING_RATE = 0.0001
    
    model = model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr = LEARNING_RATE)
    
    #metrics
    criterion = nn.CrossEntropyLoss()
    accuracy = MulticlassAccuracy(10).to(device)
    
    
    NUM_EPOCHS = 50
    
    #metric logging
    train_losses = []
    val_losses = []

    train_accuracies = []
    val_accuracies = []

    best_model_dict = None
    best_val_acc = 0

    for epoch in range(NUM_EPOCHS):
        model.train()
        train_loss = 0
        train_accuracy = 0
        for (inputs,labels) in tqdm(train_dataloader):
            optimizer.zero_grad() #zeros out the gradients accumulated from the last batch
            inputs = inputs.to(device) #place tensors on the GPU
            labels = labels.to(device) #

            logits = model(inputs) #caluclates model outputs for the minibatch
            batch_loss = criterion(logits,labels) #calculates loss
            train_loss += batch_loss.detach() #add epoch loss
            train_accuracy += accuracy(logits,labels) #add epoch accuracy

            batch_loss.backward() #calculates gradients for the whole network
            optimizer.step() #weight updates

        train_loss /= len(train_dataloader)
        train_accuracy /= len(train_dataloader)

        with torch.no_grad():#turns of gradient calculations
            model.eval() #switches models layers (e.g. BatchNorm) to evaluation mode 
            val_loss = 0
            val_accuracy = 0
            for (inputs,labels) in tqdm(test_dataloader):
                labels = labels.to(device)
                inputs = inputs.to(device)

                logits = model(inputs) 
                val_loss += criterion(logits,labels).detach() 
                val_accuracy += accuracy(logits,labels)

            val_loss /= len(test_dataloader)
            val_accuracy /= len(test_dataloader)
            if val_accuracy > best_val_acc:
                best_val_acc = val_accuracy
                best_model_dict = deepcopy(model.state_dict())


        train_losses.append(train_loss)
        val_losses.append(val_loss)

        train_accuracies.append(train_accuracy)
        val_accuracies.append(val_accuracy)

        print(f"Epoch {epoch+1}, Training Loss: {train_loss}, Validation Loss: {val_loss}")
        print(f"Training accuracy: {train_accuracy}, Validation accuracy: {val_accuracy}")
        
    ret_dict = {
        "train_losses":train_losses,
        "val_losses":val_losses,
        "train_accuracies":train_accuracies,
        "val_accuracies":val_accuracies,
        "best_model_dict":best_model_dict,
        "best_val_acc":best_val_acc
    }
    
    return ret_dict
    

    
    

In [13]:
conv_train_dict = train_loop(conv_model)


100%|██████████| 79/79 [00:01<00:00, 59.14it/s]


Epoch 1, Training Loss: 1.0734260082244873, Validation Loss: 0.30852827429771423
Training accuracy: 0.7589402794837952, Validation accuracy: 0.9211059212684631


100%|██████████| 79/79 [00:01<00:00, 60.42it/s]


Epoch 2, Training Loss: 0.2293204963207245, Validation Loss: 0.2056194245815277
Training accuracy: 0.9388031959533691, Validation accuracy: 0.9386386275291443


100%|██████████| 79/79 [00:01<00:00, 54.76it/s]


...

Epoch 48, Training Loss: 0.0028406032361090183, Validation Loss: 0.07046880573034286
Training accuracy: 0.9986240863800049, Validation accuracy: 0.9825441837310791


100%|██████████| 79/79 [00:01<00:00, 41.07it/s]


Epoch 49, Training Loss: 0.0030905811581760645, Validation Loss: 0.08632968366146088
Training accuracy: 0.9982777237892151, Validation accuracy: 0.979600191116333


100%|██████████| 79/79 [00:01<00:00, 60.08it/s]

Epoch 50, Training Loss: 0.002657767152413726, Validation Loss: 0.08485975116491318
Training accuracy: 0.9987502694129944, Validation accuracy: 0.9808436632156372


In [14]:
fc_train_dict = train_loop(fc_model)

100%|██████████| 79/79 [00:01<00:00, 61.85it/s]


Epoch 1, Training Loss: 2.293470859527588, Validation Loss: 2.2783117294311523
Training accuracy: 0.18881942331790924, Validation accuracy: 0.21592794358730316


100%|██████████| 79/79 [00:01<00:00, 62.45it/s]


Epoch 2, Training Loss: 2.25138258934021, Validation Loss: 2.2129342555999756
Training accuracy: 0.3233993649482727, Validation accuracy: 0.43284177780151367


100%|██████████| 79/79 [00:01<00:00, 62.10it/s]


...

Epoch 48, Training Loss: 0.38491007685661316, Validation Loss: 0.3715350329875946
Training accuracy: 0.890801727771759, Validation accuracy: 0.8965769410133362


100%|██████████| 79/79 [00:01<00:00, 58.78it/s]


Epoch 49, Training Loss: 0.38139501214027405, Validation Loss: 0.36840835213661194
Training accuracy: 0.8921926617622375, Validation accuracy: 0.8973777294158936


100%|██████████| 79/79 [00:01<00:00, 60.08it/s]

Epoch 50, Training Loss: 0.37823253870010376, Validation Loss: 0.3651216924190521
Training accuracy: 0.8923975229263306, Validation accuracy: 0.8979558348655701


Now we can load the best models from the state dict.

In [18]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


best_model_conv = ConvModel()
best_model_conv.load_state_dict(conv_train_dict["best_model_dict"])
best_model_conv = best_model_conv.to(device)

best_model_fc = FCModel()
best_model_fc.load_state_dict(fc_train_dict["best_model_dict"])
best_model_fc = best_model_fc.to(device)



Now we can test the models.

In [19]:
def test_model_loop(model):
    criterion = nn.CrossEntropyLoss()
    accuracy = MulticlassAccuracy(10).to(device)
    
    loss_test = 0
    accuracy_test = 0
    with torch.no_grad():
        model.eval()
        for (inputs, labels) in tqdm(test_dataloader):
            labels = labels.to(device)
            inputs = inputs.to(device)
            logits = model(inputs)
            loss_test += criterion(logits,labels).detach()
            accuracy_test += accuracy(logits,labels)
            
    loss_test /= len(test_dataloader)
    accuracy_test /= len(test_dataloader)
    
    return  loss_test, accuracy_test






In [20]:
loss_test_fc,accuracy_test_fc = test_model_loop(best_model_fc)

100%|██████████| 79/79 [00:01<00:00, 57.45it/s]


In [21]:
loss_test_conv,accuracy_test_conv = test_model_loop(best_model_conv)


100%|██████████| 79/79 [00:01<00:00, 59.24it/s]


In [23]:
print(loss_test_fc,accuracy_test_fc)
print(loss_test_conv,accuracy_test_conv)

tensor(0.3651, device='cuda:0') tensor(0.8980, device='cuda:0')
tensor(0.0562, device='cuda:0') tensor(0.9845, device='cuda:0')


In [24]:
from prettytable import PrettyTable

    
def print_out_test(loss,acc):
    with open("conv_test_metrics.txt","w") as f:
        f.write(f"Model loss:{loss.item()}, Model accuracy:{acc.item()}")
        
def print_out_metrics(train_values,val_values,metric_name,model_name):
    table = PrettyTable()
    table.field_names = ["Train "+ metric_name, "Validation "+ metric_name]

    for train_value, val_value in zip(train_values, val_values):
        table.add_row([train_value.item(), val_value.item()])
        

    with open(model_name + '_' + metric_name + '.txt', 'w') as f:
        f.write(str(table))
      
    
print_out_test(loss_test_conv,accuracy_test_conv)
print_out_metrics(conv_train_dict["train_losses"],conv_train_dict["val_losses"],"loss","convmodel")
print_out_metrics(conv_train_dict["train_accuracies"], conv_train_dict["val_accuracies"],"accuracy","convmodel")

We are going to save the best model from the training.

In [25]:
torch.save(conv_train_dict["best_model_dict"], "model.pth")